In [1]:
import bempp.api
import numpy as np
from decimal import *

import bemppUQ
from bemppUQ.config import config
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field

from bemppUQ.utils.mie import bhmie
bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'

In [2]:
k_ext, k_int = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]
polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
print("The exterior wavenumber is: {0}".format(k_ext))
print("The interior wavenumber is: {0}".format(k_int))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))


The exterior wavenumber is: 5.0
The interior wavenumber is: 6.892024376045111
----
The exterior wavelenght is: 1.2566370614359172
The exterior frequency is: 2.39E+8


In [3]:
precision = 10

h = 2.0 * np.pi / (precision * config["k_int"])

grid = bempp.api.shapes.sphere(h=h)
k_int, k_ext = config['k_int'], config['k_ext']
print(grid.leaf_view.entity_count(1))
n = k_int / k_ext
refIndex = n
numAngles = 901
s1,s2,qext,qsca,qback,gsca=bhmie(k_ext,k_int / k_ext,numAngles)
transmission_operators = assemble_operators(grid, config)
far_field, solution = evaluate_far_field(transmission_operators, config)

5997
iteration - 1 || residual - 0.234239881346 0.25908684730529785
iteration - 2 || residual - 0.0761875678721 0.13969707489013672
iteration - 3 || residual - 0.0365121680907 0.14051485061645508
iteration - 4 || residual - 0.0157389878324 0.1390068531036377
iteration - 5 || residual - 0.00711940936125 0.15422630310058594
iteration - 6 || residual - 0.00372831929687 0.13776588439941406
iteration - 7 || residual - 0.00180249823067 0.13032984733581543
iteration - 8 || residual - 0.000739818334688 0.13432574272155762
iteration - 9 || residual - 0.000350853035784 0.1491231918334961
iteration - 10 || residual - 0.000191853107259 0.13855600357055664
iteration - 11 || residual - 0.000114976579105 0.13490629196166992
iteration - 12 || residual - 7.55827697523e-05 0.14978289604187012
iteration - 13 || residual - 4.549460338e-05 0.13750147819519043
iteration - 14 || residual - 2.83799905806e-05 0.13137269020080566
iteration - 15 || residual - 1.95255401104e-05 0.1416168212890625
iteration - 16 |

In [ ]:
from matplotlib import pyplot as plt

angles = config['angles']
k_ext = config['k_ext']

fig, ax  = plt.subplots(1,1, figsize=(6,6))
A22 = far_field[2,:]   # -1j * kExt is to make agree with eqn 3.21 p70 tBohren and Huffman

plt.rcParams['figure.figsize'] = (15, 4) # Increase the figure size in the notebook

uh = abs(4 * np.pi  / (-1j * k_ext) * s1)**2
u = abs(A22[:1801])**2

ax.semilogy(angles[:1801], uh, '-r', label='BEM solution')
ax.semilogy(angles[:1801], u, '--b', label='Analytic solution')

ax.set_title(r'$\mathrm{RCS}_z(\theta)$: BEM (red) vs. analytic (blue)', fontsize=14)

ax.set_xlabel(r'$\theta$ (rad)', fontsize=14)
ax.set_ylabel(r'$\mathrm{RCS}_z(\theta)$', fontsize=14)


ymin, ymax = uh.min() * .7 , uh.max() * 1.3
ax.legend()
ax.grid()
plt.xlim(-.1, np.pi + 0.1)
plt.ylim(ymin, ymax)
plt.savefig('plots/RCSSphere.pdf')
plt.show() 

In [ ]:
precision_list = [1, 2, 5, 10, 20, 30, 40, 50,60]

for precision in precision_list:
    h = 2*np.pi/(precision*k_ext)
    print('Starting run for precision: ', precision)
    
    grid = bempp.api.shapes.sphere(h=h)
    N =  2 * grid.leaf_view.entity_count(1)
    print('Ndof: ', N)
    bempp.api.global_parameters.hmat.eps = 1E-4
    transmission_operators = assemble_operators(grid, config)
    far_field, solution = evaluate_far_field(transmission_operators, config)
    
    A22 = - 1j * k_ext * far_field[2,:]
    name = (
    "results/"
        + "precision"
    + str(precision)
    )

    my_dict = {
            "A22": A22,
            "h": h,
            "N": N,        
        }

    np.save(name + ".npy", my_dict)        

Starting run for precision:  1
Ndof:  144
iteration - 1 || residual - 0.316644872294 0.01541590690612793
iteration - 2 || residual - 0.143872340969 0.005263805389404297
iteration - 3 || residual - 0.0809891247399 0.005284786224365234
iteration - 4 || residual - 0.0541260758726 0.0042362213134765625
iteration - 5 || residual - 0.0349621772467 0.005895853042602539
iteration - 6 || residual - 0.0206817512232 0.0066471099853515625
iteration - 7 || residual - 0.0129304743124 0.006512880325317383
iteration - 8 || residual - 0.00750212308905 0.006313800811767578
iteration - 9 || residual - 0.00463504238765 0.005724430084228516
iteration - 10 || residual - 0.00297619188799 0.005371570587158203
iteration - 11 || residual - 0.00179942673382 0.006702899932861328
iteration - 12 || residual - 0.00100238373101 0.007668256759643555
iteration - 13 || residual - 0.000623106319334 0.006302833557128906
iteration - 14 || residual - 0.000335116567318 0.005700349807739258
iteration - 15 || residual - 0.0002

iteration - 9 || residual - 0.000335623142916 1.3039345741271973
iteration - 10 || residual - 0.000190714802361 1.3077266216278076
iteration - 11 || residual - 0.000139134099895 1.1921765804290771
iteration - 12 || residual - 0.000108833435371 1.0781545639038086
iteration - 13 || residual - 7.19606837846e-05 1.058417558670044
iteration - 14 || residual - 4.29782188115e-05 1.050532341003418
iteration - 15 || residual - 3.16693652281e-05 1.0629551410675049
iteration - 16 || residual - 2.24500397437e-05 1.0581507682800293
iteration - 17 || residual - 1.40080554033e-05 1.0843830108642578
iteration - 18 || residual - 1.04841362434e-05 1.0689642429351807
iteration - 19 || residual - 7.72835221179e-06 0.010032415390014648
Starting run for precision:  50
Ndof:  144438
